In [ ]:
data_directory = "nli"

X, y = [], []

x = pd.DataFrame(columns=['Text', 'Word', 'POS', 'Tag', 'Dependency'])
z = pd.DataFrame(columns = [''])

cnt = 1
for filename in os.listdir(data_directory):
    y, _ = filename.split("_")
    doc = nlp(open(os.path.join(data_directory, filename), "r").read())
    print(f'Now processing text: {filename}, text {cnt} of {len(os.listdir(data_directory))}')
    cnt += 1

    for token in doc:
        x = x.append({'Text': _, 'Word': token.text, 'POS': token.pos_, 'Tag' : token.tag_, 'Dependency' : token.dep_}, ignore_index=True)
    
Y.append(y)
#X.append(x)

print('Done')

In [1]:
import os
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

#################################################
#	Generation of the dataframe containing the 	#
#	raw training text and their assigned labels	#
#################################################

df = pd.DataFrame(columns = ['label', 'text'])

cnt = 1

data_directory = 'nli2'

for filename in os.listdir(data_directory):
    print(f'Now processing text: {filename}, text {cnt} of {len(os.listdir(data_directory))}')
    cnt += 1
    
    label_, _ = filename.split("_")
    doc_ = open(f'{data_directory}/{filename}', 'r').read()
    
    df = df.append({'label': label_, 'text': doc_}, ignore_index = True)

Now processing text: TUR_10828.txt, text 1 of 20
Now processing text: ITA_10137.txt, text 2 of 20
Now processing text: JPN_2037.txt, text 3 of 20
Now processing text: TUR_10803.txt, text 4 of 20
Now processing text: TUR_10817.txt, text 5 of 20
Now processing text: JPN_2080.txt, text 6 of 20
Now processing text: ITA_10156.txt, text 7 of 20
Now processing text: JPN_1993.txt, text 8 of 20
Now processing text: JPN_1986.txt, text 9 of 20
Now processing text: ITA_10145.txt, text 10 of 20
Now processing text: ARA_146.txt, text 11 of 20
Now processing text: JPN_2088.txt, text 12 of 20
Now processing text: ARA_154.txt, text 13 of 20
Now processing text: ARA_140.txt, text 14 of 20
Now processing text: TUR_10834.txt, text 15 of 20
Now processing text: ARA_125.txt, text 16 of 20
Now processing text: ITA_10111.txt, text 17 of 20
Now processing text: ARA_122.txt, text 18 of 20
Now processing text: ITA_10106.txt, text 19 of 20
Now processing text: TUR_10830.txt, text 20 of 20


In [2]:
df.head()

,label,text
0,TUR,"\tToday, we are living in an age of consumptio..."
1,ITA,The idea that young people nowadays do not dev...
2,JPN,"I agree with the statement as it says, the be..."
3,TUR,I do not agree that younger people enjoy life ...
4,TUR,While I was in school I was very unhappy when ...


In [3]:

df.to_csv(r'testje.csv')


In [20]:
tr = pd.read_csv('training_dataset.csv', index_col = 0)

tr.head()

,label,text
0,TUR,Life is all about the preferences. some people...
1,TEL,"\tI agree with the statement that ""Most advert..."
2,FRE,"\tIn today's society, advertisement is the sou..."
3,SPA,I am agree with the statement. I think that pe...
4,KOR,\tSome people may think that the older people ...


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [22]:
count_vect = CountVectorizer()

In [23]:
X = tr['text']
y = tr['label']

In [24]:
X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [28]:
X_train_counts = count_vect.fit_transform(X_train)

In [30]:
X_train_counts.shape

(7370, 43584)

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer

In [32]:
tfidf = TfidfTransformer()

In [33]:
X_train_tfidf = tfidf.fit_transform(X_train_counts)

In [34]:
X_train_tfidf.shape

(7370, 43584)

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vectorizer = TfidfVectorizer()

In [37]:
x_train_tfidf = vectorizer.fit_transform(X_train)

In [38]:
from sklearn.svm import LinearSVC

In [39]:
clf = LinearSVC()

In [40]:
clf.fit(X_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [41]:
from sklearn.pipeline import Pipeline

In [42]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [43]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
               

In [44]:
predictions = text_clf.predict(x_test)

In [45]:
from sklearn.metrics import confusion_matrix, classification_report

In [46]:
print(confusion_matrix(y_test, predictions))

[[239   9  11   3  14   3   7  11  20  12  17]
 [  3 249   3   1   4   1  19  29   1   2  17]
 [  8   7 256  12   7  18   0   6  12   1   1]
 [  4   6   7 274   2   8   2   2   9   2   4]
 [  7   3   1   6 240   0   1   1   4  68   3]
 [  4   7  18   6   1 269   1   3  20   2   5]
 [  1  13   5   5   1   2 222  43  11   4   6]
 [  4  31   4   3   3   1  41 228   6   7   6]
 [ 16   7  17   7   6  20   3   4 221   4  17]
 [  6   4   2   3  65   0   2   1   2 243   1]
 [ 12  11   8  12  14   5   5   9  10   3 250]]


In [47]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ARA       0.79      0.69      0.74       346
         CHI       0.72      0.76      0.74       329
         FRE       0.77      0.78      0.78       328
         GER       0.83      0.86      0.84       320
         HIN       0.67      0.72      0.69       334
         ITA       0.82      0.80      0.81       336
         JPN       0.73      0.71      0.72       313
         KOR       0.68      0.68      0.68       334
         SPA       0.70      0.69      0.69       322
         TEL       0.70      0.74      0.72       329
         TUR       0.76      0.74      0.75       339

    accuracy                           0.74      3630
   macro avg       0.74      0.74      0.74      3630
weighted avg       0.74      0.74      0.74      3630



In [51]:
import linguistic_and_stylistic_complexity

In [99]:
x = pd.read_csv('testje.csv', index_col = 0)['text']

In [100]:
x

0     \tToday, we are living in an age of consumptio...
1     The idea that young people nowadays do not dev...
2      I agree with the statement as it says, the be...
3     I do not agree that younger people enjoy life ...
4     While I was in school I was very unhappy when ...
5     I agree with the opinion. These days, In Japan...
6     Traffic, a rapid increase in fuel price and it...
7     \tIt is a difficult question, but for me, the ...
8          I definetely agree with the statement tha...
9     I am agree with this statement because if a st...
10         Every one try to look for enjoyment in hi...
11       Some people think that  it is the best way ...
12    Broad knowledge of many academic subjects than...
13    \tAlthogh, some people think that students nee...
14       Are you young? you have to know the importa...
15    In the past , people was not having any idea a...
16    If I had to refer only to my personal experien...
17      According to what all government say, th

In [103]:
import spacy    
model = spacy.load("en_core_web_sm")


In [138]:
mypath = 'ssc_data'
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [139]:
texts = pd.read_csv('testje.csv', index_col = 0)

In [180]:
cnt = 1
i = 0

for doc_ in texts['text']:
    print(f'Processing text {cnt} of {len(texts)}')
    conll = []
    doc = model(doc_)
    for sent in doc.sents:
        for token in sent:
            row = [str(token.i - sent.start), token.text, token.pos_, str(token.head.i - sent.start), token.dep_, '_']
            conll.append(row)
            if token.i + 1 == sent.end:
                conll.append(['\n'])                


    '''
    Write new file
    '''
    
    t = '\t'
    
    with open(f'{mypath}/{i}.txt', 'w+') as f: #Testen
        for line in conll:
            f.write(t.join(line))
            f.write('\n')

    cnt += 1
    i += 1

Processing text 1 of 20
Processing text 2 of 20
Processing text 3 of 20
Processing text 4 of 20
Processing text 5 of 20
Processing text 6 of 20
Processing text 7 of 20
Processing text 8 of 20
Processing text 9 of 20
Processing text 10 of 20
Processing text 11 of 20
Processing text 12 of 20
Processing text 13 of 20
Processing text 14 of 20
Processing text 15 of 20
Processing text 16 of 20
Processing text 17 of 20
Processing text 18 of 20
Processing text 19 of 20
Processing text 20 of 20


In [228]:
from linguistic_and_stylistic_complexity.bin import lascomplexity as ssc